In [ ]:
#Paquete de funciones creadas para realizar el procesado, 
#caracterización e interpolacion de los datos utilizados en la tesina

In [ ]:
#' @title Registrar archivos de origen
#' @description Funcion que crea un dataframe con los archivos de los datos de utilizados para crear el dataframe
#' @inheritParams get_raw_gpkg
#' @inheritParams get_raw_shp
#' @inheritParams get_raw_csv
#' @param raw_data_id Espera un string con el id del drive del archivo de origen
#' @param raw_data_path Espera un string con el id del drive de la ubicacion del archivo de origen
#' @param from Espera un dataframe con los archivos de origen agregados antes, si es el primero lo crea vacio por default
#' @return Retorna un dataframe que cada fila contiene un archivo de origen
#' @details Primero obtiene el dribble del archivo en el drive
#' Luego agrega una linea al dataframe con el nombre, id y ubicacion del archivo de origen
#' Se debe utilizar de manera recursiva
#' @examples from <- add_from_data(raw_data_id, raw_data_path)
#' from <- add_from_data(raw_data_id_2, raw_data_path_2, from)
def add_from_data(raw_data_id, raw_data_path, from_data=pd.DataFrame(columns=['name', 'id', 'path'])):
    #Paquetes
    import googledrive
    #Obteniendo la metadata del archivo
    dl = drive_get(as_id(raw_data_id))
    #Armado del df con los datos del archivo que necesitamos para el indice
    from_data = from_data.append({'name': dl['name'], 'id': dl['id'], 'path': raw_data_path}, ignore_index=True)
    return from_data

In [ ]:
#' @title Obtener datos crudos
#' @description Funcion utilizada para obtener los datos crudos desde el drive
#' @inheritParams get_raw_shp
#' @inheritParams get_raw_csv
#' @inheritParams get_raw_gpkg
#' @param raw_data_id Espera un string con el id del archivo de origen
#' @param temp_path Espera un String con la ruta de la carpeta temporal donde se descargan y crean los distintos archivos
#' Se recomineda usar la funcion file.path() para crear correctamente la ruta
#' @return Retorna un dribble con la ruta local de los datos crudos
#' @details Obtiene el dribble del archivo de origen y lo descarga
#' @examples datos <- get_raw_data(raw_data_id,temp_dir)
def get_raw_data(raw_data_id, temp_path):
    #Paquetes
    import googledrive
    import tibble
    #Obtiene la metadata del archivo
    metadata = drive_get(as_id(raw_data_id))
    #Carga el path
    path = file.path(temp_path, metadata['name'])
    #Si no está de manera local descarga el archivo
    try:
        drive_download(as_id(metadata['id']), path = path, overwrite = False)
    except:
        #Si el ya esta descargado solo crea el dribble
        print("El archivo ya esta descargado localmente")
    #Se agrega la ruta local al dribble
    dl = as_dribble(as_id(metadata['id']))
    dl['local_path'] = path
    return dl

In [ ]:
#' @title Obtener archivos Shapefile
#' @description Funcion utilizada para obtener los datos crudos en formato shape desde el drive
#' @param file Espera un string con el nombre del archivo a descargar
#' @param raw_data_path Espera un string con el id de la ubicacion del archivo de origen
#' @param temp_path Espera un String con la ruta de la carpeta temporal donde se descargan y crean los distintos archivos
#' Se recomineda usar la funcion file.path() para crear correctamente la ruta
#' @return Retorna una lista de dos entradas con los datos y los archivos de origen
#' @details Primero obtiene los dribbles del archivo de origen, 
#' Luego lo descarga y arma el data frame con la info de origen
#' Por ultimo importa el archivo a un sf object y arma la lista con el sf object y el df de origen
#' @examples datos <- get_raw_shp(file, raw_data_path, temp_dir)
#' @export
def get_raw_shp(file, raw_data_path, temp_path):
    #Paquetes
    import googledrive
    import dplyr
    import sf
    #Obtiene una lista de todos los archivos que hay en la carpeta del drive con los datos crudos
    lista_base = drive_ls(as_id(raw_data_path),recursive = False)
    #Filtra cada archivo del shape para obtener el id, lo descarga y arma el dataframe con los datos de origen que luego se utilizara en el indice
    #.sph
    shp = lista_base[lista_base['name'] == paste0(file,".shp")]
    shp_dl = get_raw_data(shp['id'],temp_path)
    from_data = add_from_data(shp['id'],raw_data_path)
    #.shx
    shx = lista_base[lista_base['name'] == paste0(file,".shx")]
    get_raw_data(shx['id'],temp_path)
    from_data = add_from_data(shx['id'],raw_data_path,from_data)
    #.dbf
    dbf = lista_base[lista_base['name'] == paste0(file,".dbf")]
    get_raw_data(dbf['id'],temp_path)
    from_data = add_from_data(dbf['id'],raw_data_path,from_data)
    #.prj
    prj = lista_base[lista_base['name'] == paste0(file,".prj")]
    get_raw_data(prj['id'],temp_path)
    from_data = add_from_data(prj['id'],raw_data_path,from_data)
    #Importa el shapefile descargado a un sf object, arma la lista de doble entrada y lo retorna
    data = st_read(shp_dl['local_path'])
    output = {'data': data, 'from': from_data}
    return output

In [ ]:
#' @title Obtener archivos csv
#' @description Funcion utilizada para obtener los datos crudos en formato csv desde el drive
#' @param file Espera un string con el nombre del archivo a descargar
#' @param raw_data_path Espera un string con el id de la ubicacion del archivo de origen
#' @param temp_path Espera un String con la ruta de la carpeta temporal donde se descargan y crean los distintos archivos
#' Se recomineda usar la funcion file.path() para crear correctamente la ruta
#' @return Retorna una lista de dos entradas con los datos y los archivos de origen
#' @details Primero obtiene los dribbles del archivo de origen, 
#' Luego lo descarga y arma el dframe con la info de origen
#' Por ultimo importa el archivo a un sf object y arma la lista con el sf object y el df de origen
#' @examples datos <- get_raw_shp(file, raw_data_path, temp_dir)
#' @export
def get_raw_csv(file, raw_data_path, temp_path):
    #Paquetes
    import googledrive
    import dplyr
    import sf
    #Obtiene una lista de todos los archivos que hay en la carpeta del drive con los datos crudos
    lista_base = drive_ls(as_id(raw_data_path),recursive = False)
    #Filtra el archivo csv para obtener el id, lo descarga y arma el dataframe con los datos de origen que luego se utilizara en el indice
    #.csv
    csv = lista_base[lista_base['name'] == paste0(file,".csv")]
    csv_dl = get_raw_data(csv['id'],temp_path)
    from_data = add_from_data(csv['id'],raw_data_path)
    #Importa el csv descargado a un sf object, arma la lista de doble entrada y lo retorna
    data = st_read(csv_dl['local_path'])
    output = {'data': data, 'from': from_data}
    return output

In [ ]:
#' @title Obtener archivos geopackage
#' @description Funcion utilizada para obtener los datos crudos en formato .gpkg desde el drive
#' @param file Espera un string con el nombre del archivo a descargar
#' @param raw_data_path Espera un string con el id de la ubicacion del archivo de origen
#' @param temp_path Espera un String con la ruta de la carpeta temporal donde se descargan y crean los distintos archivos
#' Se recomineda usar la funcion file.path() para crear correctamente la ruta
#' @return Retorna una lista de dos entradas con los datos y los archivos de origen
#' @details Primero obtiene los dribbles del archivo de origen, 
#' Luego lo descarga y arma el dframe con la info de origen
#' Por ultimo importa el archivo a un sf object y arma la lista con el sf object y el df de origen
#' @examples datos <- get_raw_shp(file, raw_data_path, temp_dir)
#' @export
def get_raw_gpkg(file, raw_data_path, temp_path):
    #Paquetes
    import googledrive
    import dplyr
    import sf
    #Obtiene una lista de todos los archivos que hay en la carpeta del drive con los datos crudos
    lista_base = drive_ls(as_id(raw_data_path),recursive = False)
    #Filtra el archivo gpkg para obtener el id, lo descarga y arma el dataframe con los datos de origen que luego se utilizara en el indice
    #.gpkg
    gpkg = lista_base[lista_base['name'] == paste0(file,".gpkg")]
    gpkg_dl = get_raw_data(gpkg['id'],temp_path)
    from_data = add_from_data(gpkg['id'],raw_data_path)
    #Importa el gpkg descargado a un sf object, arma la lista de doble entrada y lo retorna
    data = st_read(gpkg_dl['local_path'])
    output = {'data': data, 'from': from_data}
    return output

In [4]:
#' @title Obtener los datos del perimetro del lote
#' @description Funcion utilizada para obtener los datos del perimetro del lote cargado en el drive como gpkg
#' @param file Espera un string con el nombre del archivo a descargar
#' @param raw_data_path Espera un string con el id de la ubicacion del archivo de origen
#' @param temp_path Espera un String con la ruta de la carpeta temporal donde se descargan y crean los distintos archivos
#' Se recomineda usar la funcion file.path() para crear correctamente la ruta
#' @return Retorna un dataframe con el nombre del archivo, id de origen y la superficie
#' @details Primero obtiene los dribbles del archivo del perimetro, 
#' Luego lo descarga y importa el archivo a un sf object 
#' Por ultimo arma un dataframe con el nombre del perimetro, id en el drive y la superficie
#' @examples get_boundary_data(file, raw_data_path, temp_dir)
#' @export
def get_boundary_data(file, raw_data_path, temp_path):
    # Paquetes
    import googledrive
    import dplyr
    import sf
    import pandas as pd  # Importa pandas para trabajar con DataFrames en Python
    
    # Obtiene una lista de todos los archivos que hay en la carpeta del drive con los datos crudos
    lista_base = drive_ls(as_id(raw_data_path), recursive=False)
    
    # Filtra el archivo gpkg para obtener el id y lo descarga
    gpkg = lista_base[lista_base['name'] == file + ".gpkg"]  # Utiliza concatenación de cadenas en lugar de paste0
    gpkg_dl = get_raw_data(gpkg['id'], temp_path)
    
    # Importa el archivo descargado y arma el dataframe de retorno con el nombre del archivo,
    # el id en el drive y la superficie que abarca el perímetro
    sup = sf.read_gpkg(gpkg_dl['local_path'])  # Utiliza la función correcta para leer el archivo geoespacial
    sup_area = sup.area  # Calcula el área de los polígonos en el GeoDataFrame
    output = pd.DataFrame({'bound_file': file + ".gpkg", 'bound_id': gpkg['id'], 'bound_sup': sup_area[0]})
    
    return output

In [1]:
#' @title Cargar datos como capas de un gpkg
#' @description Funcion que carga el geopackage con cada dato en una capa y lo guarda localmente
#' @param data espera un sf object con el dato a guardar
#' @param name Espera un string con el nombre de la capa a guardar
#' @param from_df espera un dataframe con los datos de los archivos originales donde fué tomado
#' Se recomienda usar el dataframe de la lista obtenida con get_raw_shp
#' @param temp_path Espera un String con la ruta de la carpeta temporal donde se descargan y crean los distintos archivos
#' Se recomineda usar la funcion file.path() para crear correctamente la ruta
#' @param data_type espera un String con el tipo de dato a guardar para guardarlo en su correspondiente carpeta con ese nombre
#' @return Retorna un sf object pasado como parametro solo para poder usar el pipe, no tiene uso aun
#' @details Primero obtiene el directorio donde guardar los datos limpios, si no existe lo crea
#' Luego obtiene el indice, si no existe lo crea
#' Luego agrega el gpkg como una capa del tipo de dato correspondiente, si no existe el archivo lo crea
#' Por ultimo actualiza, elimina los repetidos y guarda el indice localmente
#' @examples put_gpkg_local(data, "SYM_field_01", from, "F:/Facultad/Tesina/Data processor/Temp", "mapas_rendimiento.gpkg")
#' @export
def put_gpkg_local(data, name, from_df, bound_df, temp_path, datatype):
    #Paquetes
    import dplyr
    import sf
    #Ruta local donde se guardan los datos ya procesados, si no existe la crea
    clean_path = file.path(temp_path,"clean")
    os.makedirs(clean_path, exist_ok=True)
    #Arma la ruta con la ubicación del indice
    index_path = file.path(clean_path,"index.RData")
    #Obtiene el indice
    index = pd.DataFrame(columns=['layer', 'type', 'bound_file', 'bound_id', 'bound_sup', 'from'])
    try:
        index = pd.read_pickle(index_path)
    except:
        #Si el indice no está creado, lo crea
        print("Se creo un indice nuevo")
        index.to_pickle(index_path)
    #Guarda el archivo localmente, si ya está creado le agrega una capa
    path = file.path(clean_path,datatype)
    st_write(data,path, layer=name, drive = "GPKG", delete_layer = True)
    #Actualiza el indice
    index = index.append({'layer': name, 'type': datatype, 'bound_file': bound_df['bound_file'], 'bound_id': bound_df['bound_id'], 'bound_sup': bound_df['bound_sup'], 'from.name': from_df['name'], 'from.id': from_df['id'], 'from.path': from_df['path']}, ignore_index=True)
    index = index.drop_duplicates()
    #Guarda el indice
    index.to_pickle(index_path)
    return data

In [ ]:
#' @title Medidas de resumen
#' @description Funcion que devulve las medidas de resumen de los datos de un gpkg
#' @param path Ruta del directorio donde se encuentra el archivo GPKG.
#' @param data_path  Nombre del archivo GPKG con los datos.
#' @param index_path Nombre del archivo del indice
#' Por defecto es "index.RDS"
#' @param freecor Cantidad de núcleos de CPU a dejar libres al ejecutar la función
#' Por defecto es 1
#' @param n_col Espera un entero con el numero de la columna a resumir en caso de que tenga más de una (Datos apareados)
#' Por defecto es 1
#' @param debug Este paramentro se utiliza en un entorno de prueba para cuando se quiera achicar el numero de capas a este valor
#' Por defecto es 0
#' @return Retorna un dataframe con las medidas de resumen de cada capa
#' @details Primero setea los paramentros para realizar el calculo en multiples nucleos
#' Luego lista y recorre las capas para obtener la medidas de resumen
#' Por ultimo retorna el dataframe con las medidas de cada capa
#' @examples summary_gpkg("D:/Facultad/Tesina/Temp/clean","Elevacion.gpkg",freecor = 2)
#' @export
def summary_gpkg(path, data_path, index_path="index.RDS", seed=None, outlier=True, freecor=1, ncol=1, debug=0):
    #Paquetes
    import doParallel
    import dplyr
    import sf
    #Del total de nucleos del procesador le resta los indicados y arma el cluster
    cor = detectCores()-freecor
    cl = makeCluster(cor)
    registerDoParallel(cl)
    mcaffinity(1:cor)
    on.exit(stopCluster(cl))
    
    #Obtiene una lista de todas las capas del gpkg
    layers = st_layers(file.path(path, data_path))
    #Cuando debug > 0 entramos en un entorno de prueba donde se achica el numero de layers de acuerdo al valor indicado
    if debug > 0 and debug < len(layers['name']):
        layers['name'] = layers['name'][0:debug]
    
    #Obtiene el indice almancenado de manera local
    index = pd.read_pickle(file.path(path, index_path))
    index = index.drop_duplicates(subset=['layer'])
    #Datos para crear el log
    #Crea el nombre del log de la corrida
    log_name="log_summary_gpkg.txt"
    #Ruta local donde se guardan el log, si no existe la crea
    log_path = file.path(path,"log")
    os.makedirs(log_path, exist_ok=True)
    err="1"
    #Corre los calculos en paralelo con foreach
    ret = foreach(layer = layers['name'], .combine = rbind, .packages = c("sf","dplyr")) %dopar%{
        try:
            #Levanta la capa
            #Read----
            gpkg = st_read(file.path(path, data_path),layer = layer)
            gpkg = gpkg.rename(columns={gpkg.columns[ncol]: 'data'})
            gpkg['data'] = gpkg['data'].astype(float)
            # Calcular el primer cuartil (Q1) y el tercer cuartil (Q3)
            Q1 = np.quantile(gpkg['data'], 0.25)
            Q3 = np.quantile(gpkg['data'], 0.75)
            # Calcular el rango intercuartílico (IQR)
            IQR = Q3-Q1
            # Calcular los límites inferior y superior para identificar los outliers
            limite_inferior = Q1 - 1.5 * IQR
            limite_superior = Q3 + 1.5 * IQR
            # Crear un campo logico que identifique si el dato es un outlier
            gpkg['outlier'] = np.where((gpkg['data'] < limite_inferior) | (gpkg['data'] > limite_superior), True, False)
            #Eliminando los Outliers
            if not outlier:
                gpkg = gpkg[gpkg['outlier'] == False]
            #Calculo de la distancia a todos los puntos
            #Si tiene más de 10.000 puntos hace un sample de 10.000 puntos
            if len(gpkg) > 10000:
                if seed is not None:
                    np.random.seed(seed)  # Semilla
                gpkg_s = gpkg.sample(n=10000)
            else:
                gpkg_s = gpkg
            #Obtiene desde el indice la superficie de la capa
            superficie = index['bound_sup'][index['layer'] == layer]
            #Obtiene una matriz con la distancia entre todos los puntos y se guarda el triangulo superior
            distancia = st_distance(x=gpkg_s)
            y = distancia.toarray()[np.triu_indices(distancia.shape[0])]
            x = gpkg['data'].astype(float)
            #Dataframe de retorno con todas las medidas
            ret = pd.DataFrame({'layer': layer, 'n': len(x), 'media': np.mean(x), 'mediana': np.median(x), 'min': np.min(x), 'max': np.max(x), 'iqr': IQR(x), 'sd': np.std(x), 'var': np.var(x), 'cv': np.std(x)/np.mean(x), 'asim': moments.skewness(x), 'kurt': moments.kurtosis(x), 'dist_media': np.mean(y), 'dens_m': 10000*len(gpkg)/superficie})
        except Exception as e:
            #si tira un error abre el log, registra la capa y el error, luego lo cierra
            with open(file.path(log_path,log_name), 'a') as f:
                f.write("Error en la capa: " + layer + "\n")
                f.write(err + "\n")
                f.write(str(e) + "\n")
    return ret

In [ ]:
#' @title Medidas de resumen de correlación
#' @description Funcion que devulve las medidas de resumen de la correlación de un gpkg
#' @param path Ruta del directorio donde se encuentra el archivo GPKG.
#' @param data_path  Nombre del archivo GPKG con los datos.
#' @param freecor Cantidad de núcleos de CPU a dejar libres al ejecutar la función
#' Por defecto es 1
#' @param n_col Espera un entero con el numero de la columna a resumir en caso de que tenga más de una (Datos apareados)
#' Por defecto es 1
#' @param debug Este paramentro se utiliza en un entorno de prueba para cuando se quiera achicar el numero de capas a este valor
#' Por defecto es 0
#' @return Retorna un dataframe con las medidas de resumen de cada capa
#' @details Primero setea los paramentros para realizar el calculo en multiples nucleos
#' Luego lista y recorre las capas para obtener la medidas de resumen
#' Por ultimo retorna el dataframe con las medidas de cada capa
#' @examples summary_gpkg("D:/Facultad/Tesina/Temp/clean", "Elevacion.gpkg",freecor = 2)
#' @export
def cor_summary_gpkg(path, data_path, outlier=True, freecor=1, ncol=1, debug=0):
    #Paquetes
    import doParallel
    import dplyr
    import sf
    import gstat
    import tidyverse
    cor = detectCores()-freecor
    cl = makeCluster(cor)
    registerDoParallel(cl)
    mcaffinity(1:cor)
    on.exit(stopCluster(cl))
    
    #Obtiene una lista de todas las capas del gpkg
    layers = st_layers(file.path(path, data_path))
    #Cuando debug > 0 entramos en un entorno de prueba donde se achica el numero de capas de acuerdo al valor indicado
    if debug > 0 and debug < len(layers['name']):
        layers['name'] = layers['name'][0:debug]
    
    #Obtiene el indice almancenado de manera local
    index = pd.read_pickle(file.path(path, index_path))
    index = index.drop_duplicates(subset=['layer'])
    #Datos para crear el log
    #Crea el nombre del log de la corrida
    log_name="log_summary_gpkg.txt"
    #Ruta local donde se guardan el log, si no existe la crea
    log_path = file.path(path,"log")
    os.makedirs(log_path, exist_ok=True)
    err="1"
    #Corre los calculos en paralelo con foreach
    ret = foreach(layer = layers['name'], .combine = rbind, .packages = c("sf","dplyr")) %dopar%{
        try:
            #Levanta la capa
            #Read----
            gpkg = st_read(file.path(path, data_path),layer = layer)
            gpkg = gpkg.rename(columns={gpkg.columns[ncol]: 'data'})
            gpkg['data'] = gpkg['data'].astype(float)
            # Calcular el primer cuartil (Q1) y el tercer cuartil (Q3)
            Q1 = np.quantile(gpkg['data'], 0.25)
            Q3 = np.quantile(gpkg['data'], 0.75)
            # Calcular el rango intercuartílico (IQR)
            IQR = Q3-Q1
            # Calcular los límites inferior y superior para identificar los outliers
            limite_inferior = Q1 - 1.5 * IQR
            limite_superior = Q3 + 1.5 * IQR
            # Crear un campo logico que identifique si el dato es un outlier
            gpkg['outlier'] = np.where((gpkg['data'] < limite_inferior) | (gpkg['data'] > limite_superior), True, False)
            #Eliminando los Outliers
            if not outlier:
                gpkg = gpkg[gpkg['outlier'] == False]
            #Calculo de la distancia a todos los puntos
            #Si tiene más de 10.000 puntos hace un sample de 10.000 puntos
            if len(gpkg) > 10000:
                if seed is not None:
                    np.random.seed(seed)  # Semilla
                gpkg_s = gpkg.sample(n=10000)
            else:
                gpkg_s = gpkg
            #Obtiene desde el indice la superficie de la capa
            superficie = index['bound_sup'][index['layer'] == layer]
            #Obtiene una matriz con la distancia entre todos los puntos y se guarda el triangulo superior
            distancia = st_distance(x=gpkg_s)
            y = distancia.toarray()[np.triu_indices(distancia.shape[0])]
            x = gpkg['data'].astype(float)
            #Dataframe de retorno con todas las medidas
            ret = pd.DataFrame({'layer': layer, 'n': len(x), 'media': np.mean(x), 'mediana': np.median(x), 'min': np.min(x), 'max': np.max(x), 'iqr': IQR(x), 'sd': np.std(x), 'var': np.var(x), 'cv': np.std(x)/np.mean(x), 'asim': moments.skewness(x), 'kurt': moments.kurtosis(x), 'dist_media': np.mean(y), 'dens_m': 10000*len(gpkg)/superficie})
        except Exception as e:
            #si tira un error abre el log, registra la capa y el error, luego lo cierra
            with open(file.path(log_path,log_name), 'a') as f:
                f.write("Error en la capa: " + layer + "\n")
                f.write(err + "\n")
                f.write(str(e) + "\n")
    return ret